# Anthropic Crypto Agent with Real-time News (built using Financial Modeling Prep and LlamaIndex)

## 1. Process <br>
**Crypto Analysis Agent**<br>

Overview

This notebook implements an intelligent crypto analysis system that combines Anthropic's Claude, LlamaIndex for orchestration, and Financial Modeling Prep (FMP) API for real-time cryptocurrency data and news. The agent can perform sophisticated market analysis, sentiment evaluation, and provide trading insights using natural language queries.

### Tools used:


1.   Anthropic API - Claude 3.5 Sonnett. Get your Claude API [here](https://console.anthropic.com/)
2.   LlamaIndex for function calling orchestration
3.   Financial Modeling Prep API for stock information and news. Get a free API key [here](https://site.financialmodelingprep.com/)



In [ ]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

In [ ]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate Anthropic and News API Keys

**API Key Configuration**

Instantiate Baseline LLM to be used

Store your API keys in Google Colab's user data:

ANTHROPIC_API_KEY: For Claude access
<br>FINANCIAL_MODELING_PREP_API_KEY: For market data access


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Optional: Users can swap out Claude for OpenAI models to test different models and evaluate different provider outputs / save costs



In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
FMP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

In [ ]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=CLAUDE_API_KEY)

In [103]:
openai_llm = OpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

## 3. Define Classes for Chaining Functions

Class Structure

* CryptoMetrics: Data class for storing cryptocurrency metrics
* HistoricalPrice: Data class for historical price data
* CryptoAnalyzer: Main class handling API interactions and data processing



In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Any
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

from google.colab import userdata
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

@dataclass
class CryptoMetrics:
    """Data class to store key cryptocurrency metrics"""
    symbol: str
    name: str
    price: float
    changes_percentage: float
    change: float
    day_low: float
    day_high: float
    year_high: float
    year_low: float
    market_cap: float
    moving_avg_50: float
    moving_avg_200: float
    volume: int
    avg_volume: int
    exchange: str
    open_price: float
    previous_close: float
    shares_outstanding: int
    timestamp: int

@dataclass
class HistoricalPrice:
    """Data class to store historical price data"""
    date: str
    open: float
    high: float
    low: float
    close: float
    adj_close: float
    volume: int
    unadjusted_volume: int
    change: float
    change_percent: float
    vwap: float
    label: str
    change_over_time: float

class CryptoAnalyzer:
    def __init__(self):
        """
        Initialize the Crypto analyzer using Google Colab userdata for API key.
        """
        self.api_key = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')
        if not self.api_key:
            raise ValueError("FMP API key not found in Colab userdata")

        self.base_url = "https://financialmodelingprep.com/api/v3"
        self.pro_url = "https://financialmodelingprep.com/api/v4"
        self.df = None
        self._fetch_data()

    def _fetch_data(self) -> None:
        """
        Fetch fresh cryptocurrency data from the FMP API and update the internal dataframe.
        """
        url = f"{self.base_url}/quotes/crypto?apikey={self.api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.raw_data = response.json()
            self.df = pd.DataFrame(self.raw_data)
            # Convert timestamp to datetime
            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'], unit='s')
        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to fetch crypto data: {str(e)}")
        except Exception as e:
            raise Exception(f"Error processing crypto data: {str(e)}")

    def refresh_data(self) -> None:
        """
        Manually refresh the cryptocurrency data from the API.
        """
        self._fetch_data()

    def search_crypto(self, query: str) -> List[Dict]:
        """
        Search for cryptocurrencies by symbol or name.
        Returns a list of matching cryptocurrencies.
        """
        if self.df is None:
            self._fetch_data()

        # Convert query and dataframe columns to lowercase for case-insensitive search
        query = query.lower()
        matches = self.df[
            self.df['symbol'].str.lower().str.contains(query) |
            self.df['name'].str.lower().str.contains(query)
        ]

        return matches.to_dict('records')

    def get_crypto_metrics(self, identifier: str) -> Dict:
        """
        Get detailed cryptocurrency metrics by symbol or name.
        """
        try:
            if self.df is None:
                self._fetch_data()

            # Try to find by symbol first
            crypto_data = self.df[self.df['symbol'].str.lower() == identifier.lower()]

            # If not found by symbol, try to find by name
            if crypto_data.empty:
                crypto_data = self.df[self.df['name'].str.lower() == identifier.lower()]

            if crypto_data.empty:
                return {"error": f"No cryptocurrency found for: {identifier}"}

            crypto_data = crypto_data.iloc[0]

            return {
                "symbol": crypto_data['symbol'],
                "name": crypto_data['name'],
                "price": crypto_data['price'],
                "changes_percentage": crypto_data['changesPercentage'],
                "change": crypto_data['change'],
                "day_low": crypto_data['dayLow'],
                "day_high": crypto_data['dayHigh'],
                "year_high": crypto_data['yearHigh'],
                "year_low": crypto_data['yearLow'],
                "market_cap": crypto_data['marketCap'],
                "moving_avg_50": crypto_data['priceAvg50'],
                "moving_avg_200": crypto_data['priceAvg200'],
                "volume": crypto_data['volume'],
                "avg_volume": crypto_data['avgVolume'],
                "exchange": crypto_data['exchange'],
                "open_price": crypto_data['open'],
                "previous_close": crypto_data['previousClose'],
                "shares_outstanding": crypto_data['sharesOutstanding'],
                "timestamp": crypto_data['timestamp']
            }
        except Exception as e:
            return {"error": f"Failed to get crypto metrics: {str(e)}"}

    def get_historical_prices(
        self,
        symbol: str,
        limit: Optional[int] = None,
        from_date: Optional[str] = None,
        to_date: Optional[str] = None
    ) -> Dict:
        """
        Get historical price data for a specific cryptocurrency.

        Args:
            symbol: The cryptocurrency symbol (e.g., 'BTCUSD')
            limit: Number of historical data points to retrieve (optional)
            from_date: Start date in 'YYYY-MM-DD' format (optional)
            to_date: End date in 'YYYY-MM-DD' format (optional)

        Returns:
            Dictionary containing symbol and list of historical price data
        """
        try:
            # Build URL based on provided parameters
            base_url = f"{self.base_url}/historical-price-full/{symbol}?apikey={self.api_key}"

            # Add date range parameters if provided
            if from_date and to_date:
                # Validate date formats
                try:
                    datetime.strptime(from_date, '%Y-%m-%d')
                    datetime.strptime(to_date, '%Y-%m-%d')
                except ValueError:
                    return {"error": "Invalid date format. Please use YYYY-MM-DD"}

                base_url += f"&from={from_date}&to={to_date}"
            elif limit:
                base_url += f"&limit={limit}"

            response = requests.get(base_url)
            response.raise_for_status()
            data = response.json()

            if not data or 'historical' not in data:
                return {"error": f"No historical data found for symbol: {symbol}"}

            # Convert historical data to HistoricalPrice objects
            historical_prices = []
            for entry in data['historical']:
                historical_prices.append(HistoricalPrice(
                    date=entry['date'],
                    open=entry['open'],
                    high=entry['high'],
                    low=entry['low'],
                    close=entry['close'],
                    adj_close=entry['adjClose'],
                    volume=entry['volume'],
                    unadjusted_volume=entry['unadjustedVolume'],
                    change=entry['change'],
                    change_percent=entry['changePercent'],
                    vwap=entry['vwap'],
                    label=entry['label'],
                    change_over_time=entry['changeOverTime']
                ))

            return {
                "symbol": data['symbol'],
                "historical": historical_prices,
                "period": {
                    "from": from_date if from_date else historical_prices[-1].date,
                    "to": to_date if to_date else historical_prices[0].date
                }
            }

        except requests.exceptions.RequestException as e:
            return {"error": f"API request failed: {str(e)}"}
        except Exception as e:
            return {"error": f"Failed to get historical prices: {str(e)}"}

### Helper Functions for Agent Use

### Core Functionality

**Data Retrieval Functions**

1. `get_crypto_data(symbol)`: Fetches current metrics for a cryptocurrency
2. `search_crypto(query)`: Searches for cryptocurrencies by symbol or name
3. `get_historical_crypto_data(symbol, limit, from_date, to_date)`: Retrieves historical price data
4. `get_crypto_news(symbol)`: Fetches recent news and press releases (FMP Pro Subscription required)
<br>

**Analysis Capabilities**

* Real-time price and volume metrics
* Historical price analysis
* Market sentiment analysis through news
* Comparative analysis between cryptocurrencies
* Custom valuation frameworks

#### Retrieve Quote and Profile Info - get_crypto_data




In [ ]:
# Helper Functions for Tool Use
def get_crypto_data(symbol: str) -> Dict:
    """
    Get detailed cryptocurrency metrics including price, volume, and market data.
    """
    try:
        analyzer = CryptoAnalyzer()
        return analyzer.get_crypto_metrics(symbol)
    except Exception as e:
        return {"error": f"Failed to get crypto metrics: {str(e)}"}

In [ ]:
get_crypto_data("ETHUSD")

{'symbol': 'ETHUSD',
 'name': 'Ethereum USD',
 'price': 3856.11,
 'changes_percentage': 6.58719,
 'change': 238.3114,
 'day_low': 3617.7986,
 'day_high': 3909.27,
 'year_high': 4092.2842,
 'year_low': 2113.9253,
 'market_cap': 464437569771.12,
 'moving_avg_50': 2949.163,
 'moving_avg_200': 2988.818,
 'volume': 55639565589.24167,
 'avg_volume': 22380062022.0,
 'exchange': 'CRYPTO',
 'open_price': 3617.7986,
 'previous_close': 3617.7986,
 'shares_outstanding': 120441992.0,
 'timestamp': Timestamp('2024-12-05 03:28:53')}

In [ ]:
def search_crypto(query: str) -> List[Dict]:
    """
    Search for cryptocurrencies by symbol or name.
    """
    try:
        analyzer = CryptoAnalyzer()
        return analyzer.search_crypto(query)
    except Exception as e:
        return [{"error": f"Failed to search cryptocurrencies: {str(e)}"}]

def get_crypto_data(identifier: str) -> Dict:
    """
    Get detailed cryptocurrency metrics by symbol or name.
    """
    try:
        analyzer = CryptoAnalyzer()
        return analyzer.get_crypto_metrics(identifier)
    except Exception as e:
        return {"error": f"Failed to get crypto metrics: {str(e)}"}

def get_historical_crypto_data(
    symbol: str,
    limit: Optional[int] = None,
    from_date: Optional[str] = None,
    to_date: Optional[str] = None
) -> Dict:
    """
    Get historical price data for a specific cryptocurrency.

    Args:
        symbol: The cryptocurrency symbol (e.g., 'BTCUSD')
        limit: Number of historical data points to retrieve (optional)
        from_date: Start date in 'YYYY-MM-DD' format (optional)
        to_date: End date in 'YYYY-MM-DD' format (optional)

    Example:
        # Get last 30 days of data
        btc_history = get_historical_crypto_data("BTCUSD", limit=30)

        # Get data for specific date range
        btc_history = get_historical_crypto_data(
            "BTCUSD",
            from_date="2023-08-10",
            to_date="2023-09-10"
        )
    """
    try:
        analyzer = CryptoAnalyzer()
        return analyzer.get_historical_prices(symbol, limit, from_date, to_date)
    except Exception as e:
        return {"error": f"Failed to get historical prices: {str(e)}"}

#### Get Historical Data - get_historical_crypto_data

In [ ]:
## Test Function

get_historical_crypto_data("BTCUSD")

{'symbol': 'BTCUSD',
 'historical': [HistoricalPrice(date='2024-12-04', open=95924.85, high=99263.2, low=94634.85, close=98142.15, adj_close=98142.15, volume=73304088576, unadjusted_volume=73304088576, change=2217.3, change_percent=2.3115, vwap=96991.2625, label='December 04, 24', change_over_time=0.023115),
  HistoricalPrice(date='2024-12-03', open=95862.89, high=96310.1, low=93571.3, close=95924.52, adj_close=95924.52, volume=88035654892, unadjusted_volume=88035654892, change=61.63, change_percent=0.06428974, vwap=95417.2025, label='December 03, 24', change_over_time=0.0006428974),
  HistoricalPrice(date='2024-12-02', open=97259.17, high=98200, low=94400.63, close=95862.89, adj_close=95862.89, volume=74916061117, unadjusted_volume=74916061117, change=-1396.28, change_percent=-1.43563, vwap=96430.6725, label='December 02, 24', change_over_time=-0.0143563),
  HistoricalPrice(date='2024-12-01', open=96464.95, high=97895.9, low=95753.42, close=97263.18, adj_close=97263.18, volume=4277865

#### Crypto Search Algorithm

In [ ]:
search_crypto("Bitcoin")

[{'symbol': '0XBTCUSD',
  'name': '0xBitcoin USD',
  'price': 0.09218156,
  'changesPercentage': -1.44611,
  'change': -0.00135261,
  'dayLow': 0.08973765,
  'dayHigh': 0.09453741,
  'yearHigh': 4.13419,
  'yearLow': 0.03222,
  'marketCap': 894783.3187,
  'priceAvg50': 0.17478,
  'priceAvg200': 0.39561,
  'exchange': 'CRYPTO',
  'volume': 441.4393437769034,
  'avgVolume': 97856.0,
  'open': 0.09353416,
  'previousClose': 0.09353416,
  'eps': None,
  'pe': None,
  'earningsAnnouncement': None,
  'sharesOutstanding': 9706750.0,
  'timestamp': Timestamp('2024-12-05 03:51:00')},
 {'symbol': 'BBTCUSD',
  'name': 'Baby Bitcoin USD',
  'price': 1.61e-10,
  'changesPercentage': 0.0,
  'change': 0.0,
  'dayLow': 0.0,
  'dayHigh': 0.0,
  'yearHigh': 1.61e-10,
  'yearLow': 1.61e-10,
  'marketCap': 0.0,
  'priceAvg50': 0.0,
  'priceAvg200': 0.0,
  'exchange': 'CRYPTO',
  'volume': 5.0,
  'avgVolume': nan,
  'open': 1.61e-10,
  'previousClose': 1.61e-10,
  'eps': None,
  'pe': None,
  'earningsAnno

#### Get Latest Crypto News - get_crypto_news (FMP Pro Version required)

In [ ]:
def get_crypto_news(symbol: str) -> Dict:
    """
    Get recent company news and press releases.
    """
    try:
        analyzer = CryptoAnalyzer()
        url = f"{analyzer.pro_url}/crypto_news?symbol={symbol}&limit=10&apikey={analyzer.api_key}"

        response = requests.get(url)
        news = response.json()

        if not news:
            return {"error": "No news found"}

        formatted_news = [{
            "title": item.get('title'),
            "date": item.get('publishedDate'),
            "source": item.get('site'),
            "url": item.get('url'),
            "summary": item.get('text'),
        } for item in news]

        return {"news": formatted_news}
    except Exception as e:
        return {"error": f"Failed to get company news: {str(e)}"}

In [ ]:
## Test function

get_crypto_news("MATIC")

{'news': [{'title': 'MATIC Set For Uptick: Analyst Eyes $1.30 In the Coming Weeks',
   'date': '2024-05-20T15:39:37.000Z',
   'source': 'https://tronweekly.com',
   'url': 'https://www.tronweekly.com/matic-set-for-uptick-analyst-eyes-1-30/',
   'summary': 'Polygon (MATIC) is set up for massive upward price action and has caught the eye of analysts and investors. Over the last week, MATIC has grown by almost 5%. In addition, the token is trading in an approximate horizontal channel within the medium-to-long term. The apparent result is investor indecision and a wait-and-see mode for clearer direction signals. But a a breakout from this channel would be bullish and indicate gains, and breaking the same would be bearish. The currency is currently testing resistance at the $0.72 level. Historically, trading volumes have been on the higher side for price peaks and lower with respect to price troughs, supporting strength in the currency and suggesting a break out to the upside is more likely

### Instantiate Helper Functions

---



In [ ]:
# Create Function Tools
## Comment out tools that are not relevant, chain different tools as needed

tools = {
    "crypto_metrics": FunctionTool.from_defaults(fn=get_crypto_data),
    "search_crypto": FunctionTool.from_defaults(fn=search_crypto),
    "historical_crypto_data": FunctionTool.from_defaults(fn=get_historical_crypto_data),
    "crypto_news": FunctionTool.from_defaults(fn=get_crypto_news)
}

## 4. Define Agent Flow

### Create an Anthropic Claude Agent

In [115]:
# Create the Agent
def create_crypto_agent(anthropic_llm):
    """
    Create a comprehensive crypto analysis agent with all available tools.
    """
    agent = FunctionCallingAgent.from_tools(
        list(tools.values()),
        llm=anthropic_llm,
        verbose=True,
        allow_parallel_tool_calls=False,
    )
    return agent

In [116]:
# Initialize the agent
claude_crypto_agent = create_crypto_agent(anthropic_llm)

In [117]:
# Initialize the agent
oai_crypto_agent = create_crypto_agent(openai_llm)

# Chat with an Agent

### Evaluate Different LLMs

In [110]:
# Make queries
response = oai_crypto_agent.chat("Analyze the key metrics of BTCUSD")
print(str(response))

> Running step 144a14ab-6edc-40cb-8c4d-3b3641504929. Step input: Analyze the key metrics of BTCUSD
Added user message to memory: Analyze the key metrics of BTCUSD
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "BTCUSD"}
=== Function Output ===
{'symbol': 'BTCUSD', 'name': 'Bitcoin USD', 'price': 102739.0, 'changes_percentage': 7.09106, 'change': 6802.89, 'day_low': 94581.1, 'day_high': 104015.5, 'year_high': 99655.5, 'year_low': 38521.895, 'market_cap': 2033263165752.0, 'moving_avg_50': 80404.83, 'moving_avg_200': 67208.71, 'volume': 102552971559.38351, 'avg_volume': 44089967885.0, 'exchange': 'CRYPTO', 'open_price': 95936.11, 'previous_close': 95936.11, 'shares_outstanding': 19790568.0, 'timestamp': Timestamp('2024-12-05 04:42:06')}
> Running step dfd6ba0d-4f3e-4d3a-8750-bfdc84b01755. Step input: None
=== LLM Response ===
Here are the key metrics for BTCUSD (Bitcoin USD):

- **Current Price**: $102,739.00
- **Price Change**: $6,802.89
- **Percenta

In [111]:
# Make queries
response = claude_crypto_agent.chat("Analyze the key metrics of BTCUSD")
print(str(response))

> Running step c2f8c0c4-66a1-430b-aea7-ecfc9e39c096. Step input: Analyze the key metrics of BTCUSD
Added user message to memory: Analyze the key metrics of BTCUSD
=== LLM Response ===
Certainly! I'll analyze the key metrics of BTCUSD (Bitcoin to US Dollar) for you. To do this, I'll need to use the `get_crypto_data` function to retrieve the detailed cryptocurrency metrics for Bitcoin. Let me fetch that information for you.
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "BTCUSD"}
=== Function Output ===
{'symbol': 'BTCUSD', 'name': 'Bitcoin USD', 'price': 102782.0, 'changes_percentage': 7.13588, 'change': 6845.89, 'day_low': 94581.1, 'day_high': 104015.5, 'year_high': 99655.5, 'year_low': 38521.895, 'market_cap': 2034114160176.0, 'moving_avg_50': 80404.83, 'moving_avg_200': 67208.71, 'volume': 102552971559.38351, 'avg_volume': 44089967885.0, 'exchange': 'CRYPTO', 'open_price': 95936.11, 'previous_close': 95936.11, 'shares_outstanding': 19790568.0, 't

### Test Agent Limits

In [118]:
def create_crypto_agent(anthropic_llm):
    """
    Create a comprehensive crypto analysis agent with all available tools.
    """
    agent = FunctionCallingAgent.from_tools(
        list(tools.values()),
        llm=anthropic_llm,
        verbose=True,
        allow_parallel_tool_calls=False,
    )
    return agent

# Initialize the agent
crypto_agent = create_crypto_agent(anthropic_llm)

In [119]:
response = crypto_agent.chat("Analyze Polygon's fundamentals and provide sentiment analysis around recent news")
print(str(response))

> Running step a5a66046-1225-489b-9b82-c6ccf4605a28. Step input: Analyze Polygon's fundamentals and provide sentiment analysis around recent news
Added user message to memory: Analyze Polygon's fundamentals and provide sentiment analysis around recent news
=== LLM Response ===
Certainly! I'll analyze Polygon's fundamentals and provide sentiment analysis around recent news. To do this, we'll need to use a couple of tools. First, let's get the cryptocurrency data for Polygon, and then we'll fetch recent news for sentiment analysis.

Let's start by getting the crypto data for Polygon:
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "MATIC"}
=== Function Output ===
{'error': 'No cryptocurrency found for: MATIC'}
> Running step 5762310e-578d-42ce-b76d-5c00104e4fa2. Step input: None
=== LLM Response ===
I apologize for the error. It seems that the system doesn't recognize "MATIC" as the identifier for Polygon. Let's try searching for Polygon to get the co

In [ ]:
query= "Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin"
response = crypto_agent.chat(query)
print(str(response))

> Running step 1c8c17d0-5b85-4f7a-a309-7fd3c550ad03. Step input: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin
Added user message to memory: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin
=== LLM Response ===
Certainly! I'll analyze Shiba Inu Coin (SHIB) from the perspective of a value investor like Warren Buffett or Charlie Munger. To do this, we'll need to gather some financial data about SHIB. Let's start by getting the current cryptocurrency data for Shiba Inu.
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "SHIBUSD"}
=== Function Output ===
{'symbol': 'SHIBUSD', 'name': 'Shiba Inu USD', 'price': 3.193e-05, 'changes_percentage': 11.36391, 'change': 3.258234e-06, 'day_low': 2.86e-05, 'day_high': 3.31e-05, 'year_high': 4.5e-05, 'year_low': 8e-06, 'market_cap': 18814944850.45609, 'moving_avg_50': 2.174e-0

In [ ]:
query= "Create a valuation framework for Solana (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format"
response = crypto_agent.chat(query)
print(str(response))

> Running step a28c08a8-cc1b-44e1-8273-bf147e9074fc. Step input: Create a valuation framework for Solana (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
Added user message to memory: Create a valuation framework for Solana (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
=== LLM Response ===
Certainly! I'll create a valuation framework for Solana (SOL) using a step-by-step logic to deduce a consistent method of valuing cryptocurrencies. Then, I'll present the results in a table format. Let's begin by gathering the necessary data for Solana.
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 235.81, 'changes_percentage': 0.70524, 'change': 1.65138, 'day_low': 223.17, 'day_high': 240.99, 'year_high': 263.83154, 'year_low': 60.97987, 

In [120]:
query= "Compare Polygon to Solana. What is the stronger buy for someone looking to hold long term"

response = crypto_agent.chat(query)
print(str(response))

> Running step aee03583-b98e-4f69-8239-47692e2a84ae. Step input: Compare Polygon to Solana. What is the stronger buy for someone looking to hold long term
Added user message to memory: Compare Polygon to Solana. What is the stronger buy for someone looking to hold long term
=== LLM Response ===
To compare Polygon (MATIC) to Solana (SOL) and determine which might be a stronger long-term buy, we'll need to gather data on both cryptocurrencies and analyze their fundamentals, recent performance, and ecosystem developments. Let's start by getting the data for Solana:
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 237.32, 'changes_percentage': 1.3501, 'change': 3.16138, 'day_low': 223.17, 'day_high': 240.99, 'year_high': 263.83154, 'year_low': 60.97987, 'market_cap': 112784716224.0, 'moving_avg_50': 199.2281, 'moving_avg_200': 162.5711, 'volume': 9929506813.298409, 'avg

### Testing Structured Outputs

In [ ]:
# First, define a structured query format that breaks down the analysis into clear steps
def analyze_crypto_valuation(symbol: str):
    # Step 1: Get current metrics
    metrics_query = f"Get the current market metrics for {symbol} using the get_crypto_data function"

    # Step 2: Get historical data
    historical_query = f"Get the last 30 days of historical data for {symbol} using get_historical_crypto_data function"

    # Step 3: Perform valuation analysis
    valuation_query = f"""
    Using the collected data, create a valuation framework for {symbol} by:
    1. Calculate key valuation metrics (NVT, Market Cap, TVL)
    2. Analyze historical trends
    3. Compare with industry benchmarks
    4. Provide a final valuation score
    Format the output as:
    - Key Metrics Table
    - Valuation Factors Table
    - Historical Trends Summary
    - Final Valuation Score
    """

    # Execute queries sequentially
    metrics_response = crypto_agent.chat(metrics_query)
    historical_response = crypto_agent.chat(historical_query)
    valuation_response = crypto_agent.chat(valuation_query)

    return {
        "metrics": metrics_response,
        "historical": historical_response,
        "valuation": valuation_response
    }

# Example usage:
def format_crypto_analysis(symbol: str = "SOLUSD"):
    """
    Run a complete crypto analysis with proper formatting
    """
    results = analyze_crypto_valuation(symbol)

    # Format the response using structured tables
    formatted_output = f"""
# Cryptocurrency Valuation Analysis: {symbol}

## Current Market Metrics
{results['metrics']}

## Historical Performance Analysis
{results['historical']}

## Valuation Framework
{results['valuation']}
    """

    return formatted_output

# Execute the analysis
response = format_crypto_analysis("SOLUSD")
print(response)

> Running step c1f71316-74a0-480c-8c2d-ebb4d2bb6e90. Step input: Get the current market metrics for SOLUSD using the get_crypto_data function
Added user message to memory: Get the current market metrics for SOLUSD using the get_crypto_data function
=== LLM Response ===
Certainly! I'll use the get_crypto_data function to fetch the current market metrics for SOLUSD (Solana).
=== Calling Function ===
Calling function: get_crypto_data with args: {"identifier": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 235.89, 'changes_percentage': 0.7394, 'change': 1.73138, 'day_low': 223.17, 'day_high': 240.99, 'year_high': 263.83154, 'year_low': 60.97987, 'market_cap': 112105118448.0, 'moving_avg_50': 199.2281, 'moving_avg_200': 162.5711, 'volume': 9929506813.298409, 'avg_volume': 3760214492.0, 'exchange': 'CRYPTO', 'open_price': 234.15862, 'previous_close': 234.15862, 'shares_outstanding': 475243200.0, 'timestamp': Timestamp('2024-12-05 03:38:37')}
> Running s